In [1]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [2]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]

# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN2 = r"^c20200926_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN3 = r"^c20200927_c20210812_disp_[0-9]+_[0-9]+\.txt$"
PATTERN4 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS2 = r"^c20200926_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS3 = r"^c20200927_c20210812_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS4 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"


# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

In [3]:

# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
matching_files2 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN2,0)
matching_files_ccs2 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS2,0)
matching_files3 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN3,0)
matching_files_ccs3 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS3,0)
matching_files4 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN4,0)
matching_files_ccs4 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS4,0)


# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)
print(matching_files2)
print(matching_files_ccs2)
print(matching_files3)
print(matching_files_ccs3)
print(matching_files4)
print(matching_files_ccs4)

['c20200927_c20201113_disp_58_28.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_224_108.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_388_188.txt']
['c20200927_c20201113_ccs_58_28', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_224_108', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_388_188']
['c20200926_c20201113_disp_58_28.txt', 'c20200926_c20201113_disp_140_68.txt', 'c20200926_c20201113_disp_224_108.txt', 'c20200926_c20201113_disp_306_148.txt', 'c20200926_c20201113_disp_388_188.txt']
['c20200926_c20201113_ccs_58_28', 'c20200926_c20201113_ccs_140_68', 'c20200926_c20201113_ccs_224_108', 'c20200926_c20201113_ccs_306_148', 'c20200926_c20201113_ccs_388_188']
['c20200927_c20210812_disp_58_28.txt', 'c20200927_c20210812_disp_140_68.txt', 'c20200927_c20210812_disp_224_108.txt', 'c20200927_c20210812_disp_306_148.txt', 'c20200927_c20210812_disp_388_188.txt']
['c20200927_c20210812_ccs_58_28', 'c20200927_c20210812_ccs_1

In [4]:

# load data from files into class multi-kernel
example_pairs = []
for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4]):
    datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                            matching_files,
                                            DIRECTORY_PATH_CCS,
                                            matching_files_ccs,
                                            LAT_FILE,
                                            LON_FILE,
                                            HEADING,
                                            MEAN_INC,
                                            LINES_CCS,
                                            WIDTH_CCS)
    # We need to assign some data not stored in the disp.txt files.
    datastack.get_params_from_file_name()
    datastack.get_latlon_from_file(WIDTH)
    datastack.add_lat_lon_to_data(R_START,A_START)
    datastack.crop_stack_ccs(R_STEP,A_STEP)
    # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
    # Next we add all the offset data (disp.txt) to the stack
    stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
    # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

    # add stack to list
    example_pairs.append(datastack)



In [5]:

# save_keys = ['R_idx','A_idx','Row_index','Col_index','Lon_off','Lat_off','R_off','A_off']
# for key in dir(obj[0]):
#     if ('HDBSCAN' == key[0:7]) and (key[-3:] != 'vec'):
#         save_keys.append(key)
#     if ('GLOSH' == key[0:5]) and  (key[-3:] != 'vec'):
#         save_keys.append(key)
#     if ('LOF' == key[0:3]) and  (key[-3:] != 'vec'):
#         save_keys.append(key)
# print(save_keys)
# obj[0].to_hdf5(f'./test_data/CSK_dsc/hdf5_files/{obj[0].Name[0:-4]}_outlier_detected.h5',save_keys)
objs = example_pairs[0].Stack

files = [f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[0].Name[0:-4]}_outlier_detected2.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[1].Name[0:-4]}_outlier_detected_rot.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[2].Name[0:-4]}_outlier_detected_rot.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[3].Name[0:-4]}_outlier_detected_rot.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[4].Name[0:-4]}_outlier_detected_rot.h5',
         ]
# f = h5py.File(f'./test_data/CSK_dsc/hdf5_files/{obj.Name[0:-4]}_outlier_detected.h5')
query_keys = ['HDBSCAN_labels_100_1', 'HDBSCAN_labels_100_10', 'HDBSCAN_labels_100_10_vec', 'HDBSCAN_labels_100_1_vec', 'HDBSCAN_labels_100_20', 'HDBSCAN_labels_100_20_vec', 'HDBSCAN_labels_100_30', 'HDBSCAN_labels_100_30_vec', 'HDBSCAN_labels_100_40', 'HDBSCAN_labels_100_40_vec', 'HDBSCAN_labels_100_50', 'HDBSCAN_labels_100_50_vec', 'HDBSCAN_labels_150_1', 'HDBSCAN_labels_150_15', 'HDBSCAN_labels_150_15_vec', 'HDBSCAN_labels_150_1_vec', 'HDBSCAN_labels_150_30', 'HDBSCAN_labels_150_30_vec', 'HDBSCAN_labels_150_45', 'HDBSCAN_labels_150_45_vec', 'HDBSCAN_labels_150_60', 'HDBSCAN_labels_150_60_vec', 'HDBSCAN_labels_150_75', 'HDBSCAN_labels_150_75_vec', 'HDBSCAN_labels_200_1', 'HDBSCAN_labels_200_100', 'HDBSCAN_labels_200_100_vec', 'HDBSCAN_labels_200_1_vec', 'HDBSCAN_labels_200_20', 'HDBSCAN_labels_200_20_vec', 'HDBSCAN_labels_200_40', 'HDBSCAN_labels_200_40_vec', 'HDBSCAN_labels_200_60', 'HDBSCAN_labels_200_60_vec', 'HDBSCAN_labels_200_80', 'HDBSCAN_labels_200_80_vec', 'HDBSCAN_labels_250_1', 'HDBSCAN_labels_250_100', 'HDBSCAN_labels_250_100_vec', 'HDBSCAN_labels_250_125', 'HDBSCAN_labels_250_125_vec', 'HDBSCAN_labels_250_1_vec', 'HDBSCAN_labels_250_25', 'HDBSCAN_labels_250_25_vec', 'HDBSCAN_labels_250_50', 'HDBSCAN_labels_250_50_vec', 'HDBSCAN_labels_250_75', 'HDBSCAN_labels_250_75_vec', 'HDBSCAN_labels_300_1', 'HDBSCAN_labels_300_120', 'HDBSCAN_labels_300_120_vec', 'HDBSCAN_labels_300_150', 'HDBSCAN_labels_300_150_vec', 'HDBSCAN_labels_300_1_vec', 'HDBSCAN_labels_300_30', 'HDBSCAN_labels_300_30_vec', 'HDBSCAN_labels_300_60', 'HDBSCAN_labels_300_60_vec', 'HDBSCAN_labels_300_90', 'HDBSCAN_labels_300_90_vec', 'HDBSCAN_labels_400_1', 'HDBSCAN_labels_400_120', 'HDBSCAN_labels_400_120_vec', 'HDBSCAN_labels_400_160', 'HDBSCAN_labels_400_160_vec', 'HDBSCAN_labels_400_1_vec', 'HDBSCAN_labels_400_200', 'HDBSCAN_labels_400_200_vec', 'HDBSCAN_labels_400_40', 'HDBSCAN_labels_400_40_vec', 'HDBSCAN_labels_400_80', 'HDBSCAN_labels_400_80_vec', 'HDBSCAN_labels_500_1', 'HDBSCAN_labels_500_100', 'HDBSCAN_labels_500_100_vec', 'HDBSCAN_labels_500_150', 'HDBSCAN_labels_500_150_vec', 'HDBSCAN_labels_500_1_vec', 'HDBSCAN_labels_500_200', 'HDBSCAN_labels_500_200_vec', 'HDBSCAN_labels_500_250', 'HDBSCAN_labels_500_250_vec', 'HDBSCAN_labels_500_50', 'HDBSCAN_labels_500_50_vec', 'HDBSCAN_labels_50_1', 'HDBSCAN_labels_50_10', 'HDBSCAN_labels_50_10_vec', 'HDBSCAN_labels_50_15', 'HDBSCAN_labels_50_15_vec', 'HDBSCAN_labels_50_1_vec', 'HDBSCAN_labels_50_20', 'HDBSCAN_labels_50_20_vec', 'HDBSCAN_labels_50_25', 'HDBSCAN_labels_50_25_vec', 'HDBSCAN_labels_50_5', 'HDBSCAN_labels_50_5_vec', 'HDBSCAN_outlier_scores_100_1', 'HDBSCAN_outlier_scores_100_10', 'HDBSCAN_outlier_scores_100_10_vec', 'HDBSCAN_outlier_scores_100_1_vec', 'HDBSCAN_outlier_scores_100_20', 'HDBSCAN_outlier_scores_100_20_vec', 'HDBSCAN_outlier_scores_100_30', 'HDBSCAN_outlier_scores_100_30_vec', 'HDBSCAN_outlier_scores_100_40', 'HDBSCAN_outlier_scores_100_40_vec', 'HDBSCAN_outlier_scores_100_50', 'HDBSCAN_outlier_scores_100_50_vec', 'HDBSCAN_outlier_scores_150_1', 'HDBSCAN_outlier_scores_150_15', 'HDBSCAN_outlier_scores_150_15_vec', 'HDBSCAN_outlier_scores_150_1_vec', 'HDBSCAN_outlier_scores_150_30', 'HDBSCAN_outlier_scores_150_30_vec', 'HDBSCAN_outlier_scores_150_45', 'HDBSCAN_outlier_scores_150_45_vec', 'HDBSCAN_outlier_scores_150_60', 'HDBSCAN_outlier_scores_150_60_vec', 'HDBSCAN_outlier_scores_150_75', 'HDBSCAN_outlier_scores_150_75_vec', 'HDBSCAN_outlier_scores_200_1', 'HDBSCAN_outlier_scores_200_100', 'HDBSCAN_outlier_scores_200_100_vec', 'HDBSCAN_outlier_scores_200_1_vec', 'HDBSCAN_outlier_scores_200_20', 'HDBSCAN_outlier_scores_200_20_vec', 'HDBSCAN_outlier_scores_200_40', 'HDBSCAN_outlier_scores_200_40_vec', 'HDBSCAN_outlier_scores_200_60', 'HDBSCAN_outlier_scores_200_60_vec', 'HDBSCAN_outlier_scores_200_80', 'HDBSCAN_outlier_scores_200_80_vec', 'HDBSCAN_outlier_scores_250_1', 'HDBSCAN_outlier_scores_250_100', 'HDBSCAN_outlier_scores_250_100_vec', 'HDBSCAN_outlier_scores_250_125', 'HDBSCAN_outlier_scores_250_125_vec', 'HDBSCAN_outlier_scores_250_1_vec', 'HDBSCAN_outlier_scores_250_25', 'HDBSCAN_outlier_scores_250_25_vec', 'HDBSCAN_outlier_scores_250_50', 'HDBSCAN_outlier_scores_250_50_vec', 'HDBSCAN_outlier_scores_250_75', 'HDBSCAN_outlier_scores_250_75_vec', 'HDBSCAN_outlier_scores_300_1', 'HDBSCAN_outlier_scores_300_120', 'HDBSCAN_outlier_scores_300_120_vec', 'HDBSCAN_outlier_scores_300_150', 'HDBSCAN_outlier_scores_300_150_vec', 'HDBSCAN_outlier_scores_300_1_vec', 'HDBSCAN_outlier_scores_300_30', 'HDBSCAN_outlier_scores_300_30_vec', 'HDBSCAN_outlier_scores_300_60', 'HDBSCAN_outlier_scores_300_60_vec', 'HDBSCAN_outlier_scores_300_90', 'HDBSCAN_outlier_scores_300_90_vec', 'HDBSCAN_outlier_scores_400_1', 'HDBSCAN_outlier_scores_400_120', 'HDBSCAN_outlier_scores_400_120_vec', 'HDBSCAN_outlier_scores_400_160', 'HDBSCAN_outlier_scores_400_160_vec', 'HDBSCAN_outlier_scores_400_1_vec', 'HDBSCAN_outlier_scores_400_200', 'HDBSCAN_outlier_scores_400_200_vec', 'HDBSCAN_outlier_scores_400_40', 'HDBSCAN_outlier_scores_400_40_vec', 'HDBSCAN_outlier_scores_400_80', 'HDBSCAN_outlier_scores_400_80_vec', 'HDBSCAN_outlier_scores_500_1', 'HDBSCAN_outlier_scores_500_100', 'HDBSCAN_outlier_scores_500_100_vec', 'HDBSCAN_outlier_scores_500_150', 'HDBSCAN_outlier_scores_500_150_vec', 'HDBSCAN_outlier_scores_500_1_vec', 'HDBSCAN_outlier_scores_500_200', 'HDBSCAN_outlier_scores_500_200_vec', 'HDBSCAN_outlier_scores_500_250', 'HDBSCAN_outlier_scores_500_250_vec', 'HDBSCAN_outlier_scores_500_50', 'HDBSCAN_outlier_scores_500_50_vec', 'HDBSCAN_outlier_scores_50_1', 'HDBSCAN_outlier_scores_50_10', 'HDBSCAN_outlier_scores_50_10_vec', 'HDBSCAN_outlier_scores_50_15', 'HDBSCAN_outlier_scores_50_15_vec', 'HDBSCAN_outlier_scores_50_1_vec', 'HDBSCAN_outlier_scores_50_20', 'HDBSCAN_outlier_scores_50_20_vec', 'HDBSCAN_outlier_scores_50_25', 'HDBSCAN_outlier_scores_50_25_vec', 'HDBSCAN_outlier_scores_50_5', 'HDBSCAN_outlier_scores_50_5_vec', 'HDBSCAN_probabilities_100_1', 'HDBSCAN_probabilities_100_10', 'HDBSCAN_probabilities_100_10_vec', 'HDBSCAN_probabilities_100_1_vec', 'HDBSCAN_probabilities_100_20', 'HDBSCAN_probabilities_100_20_vec', 'HDBSCAN_probabilities_100_30', 'HDBSCAN_probabilities_100_30_vec', 'HDBSCAN_probabilities_100_40', 'HDBSCAN_probabilities_100_40_vec', 'HDBSCAN_probabilities_100_50', 'HDBSCAN_probabilities_100_50_vec', 'HDBSCAN_probabilities_150_1', 'HDBSCAN_probabilities_150_15', 'HDBSCAN_probabilities_150_15_vec', 'HDBSCAN_probabilities_150_1_vec', 'HDBSCAN_probabilities_150_30', 'HDBSCAN_probabilities_150_30_vec', 'HDBSCAN_probabilities_150_45', 'HDBSCAN_probabilities_150_45_vec', 'HDBSCAN_probabilities_150_60', 'HDBSCAN_probabilities_150_60_vec', 'HDBSCAN_probabilities_150_75', 'HDBSCAN_probabilities_150_75_vec', 'HDBSCAN_probabilities_200_1', 'HDBSCAN_probabilities_200_100', 'HDBSCAN_probabilities_200_100_vec', 'HDBSCAN_probabilities_200_1_vec', 'HDBSCAN_probabilities_200_20', 'HDBSCAN_probabilities_200_20_vec', 'HDBSCAN_probabilities_200_40', 'HDBSCAN_probabilities_200_40_vec', 'HDBSCAN_probabilities_200_60', 'HDBSCAN_probabilities_200_60_vec', 'HDBSCAN_probabilities_200_80', 'HDBSCAN_probabilities_200_80_vec', 'HDBSCAN_probabilities_250_1', 'HDBSCAN_probabilities_250_100', 'HDBSCAN_probabilities_250_100_vec', 'HDBSCAN_probabilities_250_125', 'HDBSCAN_probabilities_250_125_vec', 'HDBSCAN_probabilities_250_1_vec', 'HDBSCAN_probabilities_250_25', 'HDBSCAN_probabilities_250_25_vec', 'HDBSCAN_probabilities_250_50', 'HDBSCAN_probabilities_250_50_vec', 'HDBSCAN_probabilities_250_75', 'HDBSCAN_probabilities_250_75_vec', 'HDBSCAN_probabilities_300_1', 'HDBSCAN_probabilities_300_120', 'HDBSCAN_probabilities_300_120_vec', 'HDBSCAN_probabilities_300_150', 'HDBSCAN_probabilities_300_150_vec', 'HDBSCAN_probabilities_300_1_vec', 'HDBSCAN_probabilities_300_30', 'HDBSCAN_probabilities_300_30_vec', 'HDBSCAN_probabilities_300_60', 'HDBSCAN_probabilities_300_60_vec', 'HDBSCAN_probabilities_300_90', 'HDBSCAN_probabilities_300_90_vec', 'HDBSCAN_probabilities_400_1', 'HDBSCAN_probabilities_400_120', 'HDBSCAN_probabilities_400_120_vec', 'HDBSCAN_probabilities_400_160', 'HDBSCAN_probabilities_400_160_vec', 'HDBSCAN_probabilities_400_1_vec', 'HDBSCAN_probabilities_400_200', 'HDBSCAN_probabilities_400_200_vec', 'HDBSCAN_probabilities_400_40', 'HDBSCAN_probabilities_400_40_vec', 'HDBSCAN_probabilities_400_80', 'HDBSCAN_probabilities_400_80_vec', 'HDBSCAN_probabilities_500_1', 'HDBSCAN_probabilities_500_100', 'HDBSCAN_probabilities_500_100_vec', 'HDBSCAN_probabilities_500_150', 'HDBSCAN_probabilities_500_150_vec', 'HDBSCAN_probabilities_500_1_vec', 'HDBSCAN_probabilities_500_200', 'HDBSCAN_probabilities_500_200_vec', 'HDBSCAN_probabilities_500_250', 'HDBSCAN_probabilities_500_250_vec', 'HDBSCAN_probabilities_500_50', 'HDBSCAN_probabilities_500_50_vec', 'HDBSCAN_probabilities_50_1', 'HDBSCAN_probabilities_50_10', 'HDBSCAN_probabilities_50_10_vec', 'HDBSCAN_probabilities_50_15', 'HDBSCAN_probabilities_50_15_vec', 'HDBSCAN_probabilities_50_1_vec', 'HDBSCAN_probabilities_50_20', 'HDBSCAN_probabilities_50_20_vec', 'HDBSCAN_probabilities_50_25', 'HDBSCAN_probabilities_50_25_vec', 'HDBSCAN_probabilities_50_5', 'HDBSCAN_probabilities_50_5_vec', 'LOF_labels_100', 'LOF_labels_100_vec', 'LOF_labels_150', 'LOF_labels_150_vec', 'LOF_labels_200', 'LOF_labels_200_vec', 'LOF_labels_250', 'LOF_labels_250_vec', 'LOF_labels_300', 'LOF_labels_300_vec', 'LOF_labels_400', 'LOF_labels_400_vec', 'LOF_labels_50', 'LOF_labels_500', 'LOF_labels_500_vec', 'LOF_labels_50_vec', 'LOF_outlier_score_100_vec', 'LOF_outlier_score_150_vec', 'LOF_outlier_score_200_vec', 'LOF_outlier_score_250_vec', 'LOF_outlier_score_300_vec', 'LOF_outlier_score_400_vec', 'LOF_outlier_score_500_vec', 'LOF_outlier_score_50_vec', 'LOF_outlier_scores_100', 'LOF_outlier_scores_150', 'LOF_outlier_scores_200', 'LOF_outlier_scores_250', 'LOF_outlier_scores_300', 'LOF_outlier_scores_400', 'LOF_outlier_scores_50', 'LOF_outlier_scores_500', 'Mag_off_med_diff_2', 'Mag_off_med_diff_2_vec', 'Mag_off_med_diff_3', 'Mag_off_med_diff_3_vec', 'Mag_off_med_diff_4', 'Mag_off_med_diff_4_vec', 'Mag_off_med_diff_5', 'Mag_off_med_diff_5_vec', 'Mag_off_med_diff_6', 'Mag_off_med_diff_6_vec', 'Mag_off_med_diff_7', 'Mag_off_med_diff_7_vec', 'Mag_off_med_diff_8', 'Mag_off_med_diff_8_vec', 'Mag_off_med_diff_9', 'Mag_off_med_diff_9_vec']

for (obj,file) in zip(objs,files):
    print(obj,file)
    obj.from_hdf5(file,query_keys)

<SPOTSAR_main.Post_processing.singlekernel.SingleKernel object at 0x7fe1c3e7bbe0> ./test_data/CSK_dsc/hdf5_files/c20200927_c20201113_disp_58_28_outlier_detected2.h5


<SPOTSAR_main.Post_processing.singlekernel.SingleKernel object at 0x7fe19cf292e0> ./test_data/CSK_dsc/hdf5_files/c20200927_c20201113_disp_140_68_outlier_detected_rot.h5
<SPOTSAR_main.Post_processing.singlekernel.SingleKernel object at 0x7fe19cf299a0> ./test_data/CSK_dsc/hdf5_files/c20200927_c20201113_disp_224_108_outlier_detected_rot.h5
<SPOTSAR_main.Post_processing.singlekernel.SingleKernel object at 0x7fe19cf2f160> ./test_data/CSK_dsc/hdf5_files/c20200927_c20201113_disp_306_148_outlier_detected_rot.h5
<SPOTSAR_main.Post_processing.singlekernel.SingleKernel object at 0x7fe19cf2f970> ./test_data/CSK_dsc/hdf5_files/c20200927_c20201113_disp_388_188_outlier_detected_rot.h5


In [6]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

In [7]:
%matplotlib osx
import pickle
import itertools
import random

from sklearn import metrics
from matplotlib.collections import LineCollection
plt.close('all')

# plotting conditions
plotting = 0
plotting_flag = 0
labels = 0
verbose= False

# dataset to test
obj = example_pairs[0].Stack[1]

min_samples_facts = [0.00001, 0.1, 0.2, 0.3, 0.4, 0.5]
min_cluster_sizes = [50,100,150,200,250,300,400,500]
filter_radius = [2,3,4,5,6,7,8,9]

n_cat = 50 # number of samples from each catergory of inlier/outlier
n_runs = 1 # number of runs

# get data
r_win = getattr(obj, "R_win")
a_win = getattr(obj, "A_win")
r_idx = getattr(obj, "R_idx")
a_idx = getattr(obj, "A_idx")

# get testing data
test_data1 = pd.read_csv(
            f"./test_data/CSK_dsc/{obj.Name[:-4]}_test_set.csv", header=0
        ).to_numpy()
test_data2 = pd.read_csv(
            f"./test_data/CSK_dsc/{obj.Name[:-4]}_test_set3.csv", header=0
        ).to_numpy()

test_data3 = pd.read_csv(
            f"./test_data/CSK_dsc/{obj.Name[:-4]}_test_set4.csv", header=0
        ).to_numpy()

# select which test data to use
test_data4 = np.row_stack((test_data1,test_data2,test_data3))

test_data4 = np.unique(test_data4,axis=0)


In [8]:
# seperate based on category
# n_cat = 60
# test_data3 = test_data3[test_data3[:,3]==0]
sel0 = test_data4[test_data4[:,2]==0]
sel1 = test_data4[test_data4[:,2]==1]
sel2 = test_data4[test_data4[:,2]==2]
sel3 = test_data4[test_data4[:,2]==3]

# initialize outlut lists
auc_list = []
fpr_list = []
tpr_list = []
thresh_list = []

auc_mag_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_med_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_hdb_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_glsh_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_lof_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_comb123_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_comb23_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
# main loop
for run_id in range(n_runs):
    ax_id = 0
    # select n_cat point from each category of the testing data

    test_data = np.row_stack((sel0[random.sample(range(np.shape(sel0)[0]),k=n_cat),:],
                            sel1[random.sample(range(np.shape(sel1)[0]),k=n_cat),:],
                            sel2[random.sample(range(np.shape(sel2)[0]),k=n_cat),:],
                            sel3[random.sample(range(np.shape(sel3)[0]),k=n_cat),:]))
    # run for each minimum cluster size
    for clust_id, (min_clust_size, filt_rad) in enumerate(zip(min_cluster_sizes,filter_radius)):
        min_samples = [np.max([1, int(np.round(min_clust_size * min_samples_fact))]) for min_samples_fact in min_samples_facts]
        
        for samp_id, min_sample in enumerate(min_samples):
            
            # go through each method and collect relevant results
            print(run_id, ax_id)
            HDBSCAN_labels = getattr(
                obj, f"HDBSCAN_probabilities_{min_clust_size}_{min_sample}"
            )
            GLOSH_labels = getattr(
                obj, f"HDBSCAN_outlier_scores_{min_clust_size}_{min_sample}"
            )
            LOF_labels = getattr(obj, f"LOF_outlier_scores_{min_clust_size}")
            Mag_labels = getattr(obj, "Mag")
            Med_labels = getattr(obj, f"Mag_off_med_diff_{filt_rad}")

            # get coordinates and offsets for plotting vectors of testing data
            all_lons=getattr(obj,'Lon_off')
            all_lats=getattr(obj,'Lat_off')
            all_X_off=getattr(obj,'X_off')
            all_Y_off=getattr(obj,'Y_off')

            # get testing data
            test_labels = -1 * np.ones(np.shape(test_data[:, 1]))
            mask = (test_data[:, 2] > 1)
            test_labels[mask] = 1  # all inliers
            test_lats = np.empty_like(test_labels)
            test_lons = np.empty_like(test_labels)
            test_X_off = np.empty_like(test_labels)
            test_Y_off = np.empty_like(test_labels)
            for row_id,test_vec in enumerate(test_data):
                
                idx = np.argwhere((r_idx == int(test_vec[0])) & (a_idx == int(test_vec[1])))
                test_lats[row_id] = all_lats[idx[0][0], idx[0][1]]
                test_lons[row_id] = all_lons[idx[0][0], idx[0][1]]
                test_X_off[row_id] = all_X_off[idx[0][0], idx[0][1]]
                test_Y_off[row_id] = all_Y_off[idx[0][0], idx[0][1]]
            
            if plotting == 0:
                fig0, ax0 = plt.subplots(1,1)
                ax0.hist(test_data[:,2],4)
                fig1, ax = plt.subplots(1,1)
                ax.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_EXTENT)
                q = ax.quiver(test_lons,test_lats,
                                test_X_off,test_Y_off,
                                test_data[:,2],
                                scale=50, 
                                width = 0.008, 
                                edgecolor='black',
                                linewidth=0.2)
                plotting = 1

            # collect results of methods for testing data points
            Mag_test_labels = np.full(np.shape(test_labels), np.nan)
            Med_test_labels = np.full(np.shape(test_labels), np.nan)
            HDBSCAN_test_labels = np.full(np.shape(test_labels), np.nan)
            GLOSH_test_labels = np.full(np.shape(test_labels), np.nan)
            LOF_test_labels = np.full(np.shape(test_labels), np.nan)
            for row_id, test_vec in enumerate(test_data):
                idx = np.argwhere((r_idx == int(test_vec[0])) & (a_idx == int(test_vec[1])))
                Mag_test_labels[row_id] = Mag_labels[idx[0][0], idx[0][1]]
                Med_test_labels[row_id] = Med_labels[idx[0][0], idx[0][1]]
                HDBSCAN_test_labels[row_id] = HDBSCAN_labels[idx[0][0], idx[0][1]]
                GLOSH_test_labels[row_id] = GLOSH_labels[idx[0][0], idx[0][1]]
                LOF_test_labels[row_id] = LOF_labels[idx[0][0], idx[0][1]]


            ## convert metric to inlier probability ##

            # magnitude filter (1 - normalised magnitude)
            p_in_mag = 1- (Mag_test_labels-np.nanmin(Mag_labels))/(np.nanmax(Mag_labels)-np.nanmin(Mag_labels))

            # Median filter (1 - max normalised median difference)
            p_in_med = 1- (Med_test_labels/(np.nanmax(Med_labels)))

            # LOF (normalised negative LOF)
            p_in_lof = (LOF_test_labels-np.nanmin(LOF_labels))/(np.nanmax(LOF_labels)-np.nanmin(LOF_labels))

            # HDBSCAN (probability of belonging to cluster (outliers have probability 0))
            p_in_hdb = HDBSCAN_test_labels

            # GLOSH (1 - outlier probability)
            p_in_glsh = 1-GLOSH_test_labels


            # p_in_comb123 = 1 - ((1-p_in_lof)*(1-p_in_glsh)*(1-p_in_hdb))
            # p_in_comb23 = 1 - ((1-p_in_glsh)*(1-p_in_hdb))

            p_in_comb123 = (p_in_lof + p_in_glsh + p_in_hdb)/3
            # p_in_comb23 = (p_in_hdb + p_in_glsh)/2

            p_in_comb23 = (p_in_hdb + p_in_med)/2
            # p_in_comb23 = 1 - ((1-p_in_med)*(1-p_in_hdb))



            # magnitude
            fpr_mag, tpr_mag, thresh_mag = metrics.roc_curve(
                test_labels[~np.isnan(p_in_mag)],
                p_in_mag[~np.isnan(p_in_mag)],
            )
            print(f'thresh mag [0]: {thresh_mag[0]} {thresh_mag[-1]}')
            auc_mag = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_mag)],
                p_in_mag[~np.isnan(p_in_mag)],
            )

            # magnitude
            fpr_med, tpr_med, thresh_med = metrics.roc_curve(
                test_labels[~np.isnan(p_in_med)],
                p_in_med[~np.isnan(p_in_med)],
            )
            print(f'thresh med [0]: {thresh_med[0]} {thresh_med[-1]}')
            auc_med = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_med)],
                p_in_med[~np.isnan(p_in_med)],
            )
            
            # HDBSCAN
            fpr_hdb, tpr_hdb, thresh_hdb = metrics.roc_curve(
                test_labels[~np.isnan(p_in_hdb)],
                p_in_hdb[~np.isnan(p_in_hdb)],
            )
            print(f'thresh hdb [0]: {thresh_hdb[0]} {thresh_hdb[-1]}')
            auc_hdb = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_hdb)],
                p_in_hdb[~np.isnan(p_in_hdb)],
            )
            store_hdb_1 = (fpr_hdb, tpr_hdb, thresh_hdb,auc_hdb)

            # GLOSH
            fpr_glsh, tpr_glsh, thresh_glsh = metrics.roc_curve(
                test_labels[~np.isnan(p_in_glsh)],
                p_in_glsh[~np.isnan(p_in_glsh)],
            )
            print(f'thresh glsh [0]: {thresh_glsh[0]} {thresh_glsh[-1]}')
            auc_glsh = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_glsh)],
                p_in_glsh[~np.isnan(p_in_glsh)],
            )

            # LOF
            fpr_lof, tpr_lof, thresh_lof = metrics.roc_curve(
                test_labels[~np.isnan(p_in_lof)],
                p_in_lof[~np.isnan(p_in_lof)],
            )
            print(f'thresh lof [0]: {thresh_lof[0]} {thresh_lof[-1]}')
            auc_lof = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_lof)],
                p_in_lof[~np.isnan(p_in_lof)],
            )

            # comb123
            fpr_comb123, tpr_comb123, thresh_comb123 = metrics.roc_curve(
                test_labels[~np.isnan(p_in_comb123)],
                p_in_comb123[~np.isnan(p_in_comb123)],
            )
            print(f'thresh comb123 [0]: {thresh_comb123[0]} {thresh_comb123[-1]}')
            auc_comb123 = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_comb123)],
                p_in_comb123[~np.isnan(p_in_comb123)],
            )

            # comb23
            fpr_comb23, tpr_comb23, thresh_comb23 = metrics.roc_curve(
                test_labels[~np.isnan(p_in_comb23)],
                p_in_comb23[~np.isnan(p_in_comb23)],
            )
            print(f'thresh comb23 [0]: {thresh_comb23[0]} {thresh_comb23[-1]}')
            auc_comb23 = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_comb23)],
                p_in_comb23[~np.isnan(p_in_comb23)],
            )
            

            if verbose:
                print(f'{ax_id} M min, max:{np.min(thresh_mag)},{np.max(thresh_mag)}')
                print(f'{ax_id} Med min, max:{np.min(thresh_med)},{np.max(thresh_med)}')
                print(f'{ax_id} H min, max:{np.min(thresh_hdb)},{np.max(thresh_hdb)}')
                print(f'{ax_id} G min, max:{np.min(thresh_glsh)},{np.max(thresh_glsh)}')
                print(f'{ax_id} L min, max:{np.min(thresh_lof)},{np.max(thresh_lof)}')
                print(f'{ax_id} C123 min, max:{np.min(thresh_comb123)},{np.max(thresh_comb123)}')
                print(f'{ax_id} C23 min, max:{np.min(thresh_comb23)},{np.max(thresh_comb23)}')

                print(f'{ax_id} M min, max:{np.nanmin(p_in_mag)},{np.nanmax(p_in_mag)}')
                print(f'{ax_id} Med min, max:{np.nanmin(p_in_med)},{np.nanmax(p_in_med)}')
                print(f'{ax_id} H min, max:{np.nanmin(p_in_hdb)},{np.nanmax(p_in_hdb)}')
                print(f'{ax_id} G min, max:{np.nanmin(p_in_glsh)},{np.nanmax(p_in_glsh)}')
                print(f'{ax_id} L min, max:{np.nanmin(p_in_lof)},{np.nanmax(p_in_lof)}')
                print(f'{ax_id} C123 min, max:{np.nanmin(p_in_comb123)},{np.nanmax(p_in_comb123)}')
                print(f'{ax_id} C23 min, max:{np.nanmin(p_in_comb23)},{np.nanmax(p_in_comb23)}')

            # set thresholds that are above the maximum value in the test dataset to the max value of the test dataset
            thresh_mag[thresh_mag > np.nanmax(p_in_mag)] = np.nanmax(p_in_mag)
            thresh_med[thresh_med > np.nanmax(p_in_med)] = np.nanmax(p_in_med)
            thresh_hdb[thresh_hdb > np.nanmax(p_in_hdb)] = np.nanmax(p_in_hdb)
            thresh_glsh[thresh_glsh > np.nanmax(p_in_glsh)] = np.nanmax(p_in_glsh)
            thresh_lof[thresh_lof > np.nanmax(p_in_lof)] = np.nanmax(p_in_lof)
            thresh_comb123[thresh_comb123 > np.nanmax(p_in_comb123)] = np.nanmax(p_in_comb123)
            thresh_comb23[thresh_comb23 > np.nanmax(p_in_comb23)] = np.nanmax(p_in_comb23)
            # store true/false positive rate, auc values and thresholds for plotting and calculatign averages
            fpr_stack = [fpr_mag,fpr_med,fpr_hdb,fpr_glsh,fpr_lof,fpr_comb123,fpr_comb23]
            tpr_stack = [tpr_mag,tpr_med,tpr_hdb,tpr_glsh,tpr_lof,tpr_comb123,tpr_comb23]
            auc_stack = [auc_mag,auc_med,auc_hdb,auc_glsh,auc_lof,auc_comb123,auc_comb23]
            thresh_stack = [thresh_mag,thresh_med,thresh_hdb,thresh_glsh,thresh_lof,thresh_comb123,thresh_comb23]
            fpr_list.append(fpr_stack)
            tpr_list.append(tpr_stack)
            thresh_list.append(thresh_stack)
            auc_mag_mat[clust_id,samp_id,run_id] = auc_mag
            auc_med_mat[clust_id,samp_id,run_id] = auc_med
            auc_hdb_mat[clust_id,samp_id,run_id] = auc_hdb
            auc_glsh_mat[clust_id,samp_id,run_id] = auc_glsh
            auc_lof_mat[clust_id,samp_id,run_id] = auc_lof
            auc_comb123_mat[clust_id,samp_id,run_id] = auc_comb123
            auc_comb23_mat[clust_id,samp_id,run_id] = auc_comb23

            # increment ax_id
            ax_id += 1


print(np.shape(fpr_list))        
# get average of auc values across runs
mean_mag = np.mean(auc_mag_mat,axis=2)
mean_med = np.mean(auc_med_mat,axis=2)
mean_hdb = np.mean(auc_hdb_mat,axis=2)
mean_glsh = np.mean(auc_glsh_mat,axis=2)
mean_lof = np.mean(auc_lof_mat,axis=2)
mean_comb123 = np.mean(auc_comb123_mat,axis=2)
mean_comb23 = np.mean(auc_comb23_mat,axis=2)

ax_id = 0
fpr_mag_p = np.array(fpr_list)[:,0]
fpr_med_p = np.array(fpr_list)[:,1]
fpr_hdb_p = np.array(fpr_list)[:,2]
fpr_glsh_p = np.array(fpr_list)[:,3]
fpr_lof_p = np.array(fpr_list)[:,4]
fpr_comb123_p = np.array(fpr_list)[:,5]
fpr_comb23_p = np.array(fpr_list)[:,6]

tpr_mag_p = np.array(tpr_list)[:,0]
tpr_med_p = np.array(tpr_list)[:,1]
tpr_hdb_p = np.array(tpr_list)[:,2]
tpr_glsh_p = np.array(tpr_list)[:,3]
tpr_lof_p = np.array(tpr_list)[:,4]
tpr_comb123_p = np.array(tpr_list)[:,5]
tpr_comb23_p = np.array(tpr_list)[:,6]

thresh_mag_p = np.array(thresh_list,dtype='object')[:,0]
thresh_med_p = np.array(thresh_list,dtype='object')[:,1]
thresh_hdb_p = np.array(thresh_list,dtype='object')[:,2]
thresh_glsh_p = np.array(thresh_list,dtype='object')[:,3]
thresh_lof_p = np.array(thresh_list,dtype='object')[:,4]
thresh_comb123_p = np.array(thresh_list,dtype='object')[:,5]
thresh_comb23_p = np.array(thresh_list,dtype='object')[:,6]



0 0


thresh mag [0]: 1.9997039697155095 0.5131676403979505
thresh med [0]: 2.0 0.4332487341093313
thresh hdb [0]: 2.0 0.0
thresh glsh [0]: 2.0 0.29335357346661384
thresh lof [0]: 1.9985985145789198 0.18881567544213365
thresh comb123 [0]: 1.9979493292535961 0.2725087702538534
thresh comb23 [0]: 2.0 0.21662436705466565
0 1
thresh mag [0]: 1.9997039697155095 0.5131676403979505
thresh med [0]: 2.0 0.4332487341093313
thresh hdb [0]: 2.0 0.0
thresh glsh [0]: 2.0 0.29335357346661384
thresh lof [0]: 1.9985985145789198 0.18881567544213365
thresh comb123 [0]: 1.9987720128968838 0.2637515139104543
thresh comb23 [0]: 2.0 0.21662436705466565
0 2
thresh mag [0]: 1.9997039697155095 0.5131676403979505
thresh med [0]: 2.0 0.4332487341093313
thresh hdb [0]: 2.0 0.0
thresh glsh [0]: 2.0 0.1533341959149498
thresh lof [0]: 1.9985985145789198 0.18881567544213365
thresh comb123 [0]: 1.9987720128968838 0.18313258764861315
thresh comb23 [0]: 2.0 0.21662436705466565
0 3
thresh mag [0]: 1.9997039697155095 0.513167640

/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_48203/929587775.py:264: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fpr_mag_p = np.array(fpr_list)[:,0]
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_48203/929587775.py:265: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with diffe

In [9]:
# make copies of thresh_xxx_p
import copy
thresh_mag_p_cp = copy.deepcopy(thresh_mag_p)
thresh_med_p_cp = copy.deepcopy(thresh_med_p)
thresh_hdb_p_cp = copy.deepcopy(thresh_hdb_p)
thresh_glsh_p_cp = copy.deepcopy(thresh_glsh_p)
thresh_lof_p_cp = copy.deepcopy(thresh_lof_p)
thresh_comb123_p_cp = copy.deepcopy(thresh_comb123_p)
thresh_comb23_p_cp = copy.deepcopy(thresh_comb23_p)
tpr_mag_p_cp = copy.deepcopy(tpr_mag_p)
tpr_med_p_cp = copy.deepcopy(tpr_med_p)
tpr_hdb_p_cp = copy.deepcopy(tpr_hdb_p)
tpr_glsh_p_cp = copy.deepcopy(tpr_glsh_p)
tpr_lof_p_cp = copy.deepcopy(tpr_lof_p)
tpr_comb123_p_cp = copy.deepcopy(tpr_comb123_p)
tpr_comb23_p_cp = copy.deepcopy(tpr_comb23_p)
fpr_mag_p_cp = copy.deepcopy(fpr_mag_p)
fpr_med_p_cp = copy.deepcopy(fpr_med_p)
fpr_hdb_p_cp = copy.deepcopy(fpr_hdb_p)
fpr_glsh_p_cp = copy.deepcopy(fpr_glsh_p)
fpr_lof_p_cp = copy.deepcopy(fpr_lof_p)
fpr_comb123_p_cp = copy.deepcopy(fpr_comb123_p)
fpr_comb23_p_cp = copy.deepcopy(fpr_comb23_p)
sizes = (np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs)
opt_thresh_mag = np.full(sizes,np.nan)
opt_thresh_med = np.full(sizes,np.nan)
opt_thresh_hdb = np.full(sizes,np.nan)
opt_thresh_glsh = np.full(sizes,np.nan)
opt_thresh_lof = np.full(sizes,np.nan)
opt_thresh_comb123 = np.full(sizes,np.nan)
opt_thresh_comb23 = np.full(sizes,np.nan)


In [10]:

fontsize=16
plt.rcParams.update({'font.size': fontsize})

plot_id = 0
for run_id in range(n_runs):
    ax_id = 0
    for clust_id in range(np.size(min_cluster_sizes)):
        for samp_id in range(np.size(min_samples_facts)):
            print(plot_id)
            print(f'max pos M: {np.argmax(tpr_mag_p_cp[plot_id]-fpr_mag_p_cp[plot_id])} : {thresh_mag_p_cp[plot_id][np.argmax(tpr_mag_p_cp[plot_id]-fpr_mag_p_cp[plot_id])]}')
            print(f'max pos Med: {np.argmax(tpr_med_p_cp[plot_id]-fpr_med_p_cp[plot_id])} : {thresh_med_p_cp[plot_id][np.argmax(tpr_med_p_cp[plot_id]-fpr_med_p_cp[plot_id])]}')
            print(f'max pos H: {np.argmax(tpr_hdb_p_cp[plot_id]-fpr_hdb_p_cp[plot_id])} : {thresh_hdb_p_cp[plot_id][np.argmax(tpr_hdb_p_cp[plot_id]-fpr_hdb_p_cp[plot_id])]}')
            print(f'max pos G: {np.argmax(tpr_glsh_p_cp[plot_id]-fpr_glsh_p_cp[plot_id])} : {thresh_glsh_p_cp[plot_id][np.argmax(tpr_glsh_p_cp[plot_id]-fpr_glsh_p_cp[plot_id])]}')
            print(f'max pos L: {np.argmax(tpr_lof_p_cp[plot_id]-fpr_lof_p_cp[plot_id])} : {thresh_lof_p_cp[plot_id][np.argmax(tpr_lof_p_cp[plot_id]-fpr_lof_p_cp[plot_id])]}')
            print(f'max pos C123: {np.argmax(tpr_comb123_p_cp[plot_id]-fpr_comb123_p_cp[plot_id])} : {thresh_comb123_p_cp[plot_id][np.argmax(tpr_comb123_p_cp[plot_id]-fpr_comb123_p_cp[plot_id])]}')
            print(f'max pos C23: {np.argmax(tpr_comb23_p_cp[plot_id]-fpr_comb23_p_cp[plot_id])} : {thresh_comb23_p_cp[plot_id][np.argmax(tpr_comb23_p_cp[plot_id]-fpr_comb23_p_cp[plot_id])]}')
            opt_thresh_mag[clust_id,samp_id,run_id] = thresh_mag_p_cp[plot_id][np.argmax(tpr_mag_p_cp[plot_id]-fpr_mag_p_cp[plot_id])]
            opt_thresh_med[clust_id,samp_id,run_id] = thresh_med_p_cp[plot_id][np.argmax(tpr_med_p_cp[plot_id]-fpr_med_p_cp[plot_id])]
            opt_thresh_hdb[clust_id,samp_id,run_id] = thresh_hdb_p_cp[plot_id][np.argmax(tpr_hdb_p_cp[plot_id]-fpr_hdb_p_cp[plot_id])]
            opt_thresh_glsh[clust_id,samp_id,run_id] = thresh_glsh_p_cp[plot_id][np.argmax(tpr_glsh_p_cp[plot_id]-fpr_glsh_p_cp[plot_id])]
            opt_thresh_lof[clust_id,samp_id,run_id] = thresh_lof_p_cp[plot_id][np.argmax(tpr_lof_p_cp[plot_id]-fpr_lof_p_cp[plot_id])]
            opt_thresh_comb123[clust_id,samp_id,run_id] = thresh_comb123_p_cp[plot_id][np.argmax(tpr_comb123_p_cp[plot_id]-fpr_comb123_p_cp[plot_id])]
            opt_thresh_comb23[clust_id,samp_id,run_id] = thresh_comb23_p_cp[plot_id][np.argmax(tpr_comb23_p_cp[plot_id]-fpr_comb23_p_cp[plot_id])]
            plot_id +=1
# opt_thresh = [opt_thresh_mag, opt_thresh_hdb, opt_thresh_glsh, opt_thresh_lof]


min_samp_label = [ str(i) for i in min_samples_facts]
min_samp_label[0] = '>0'
# min_clust_label_facts = [str(i) for i in min_clust_facts]
min_clust_label_facts = [str(i) for i in min_cluster_sizes]
min_clust_label = [str(i) for i in min_cluster_sizes]

# print(opt_thresh_glsh[:,:,1])
print(f'max thresh M: {np.max(opt_thresh_mag)}')
print(f'max thresh Med: {np.max(opt_thresh_med)}')
print(f'max thresh H: {np.max(opt_thresh_hdb)}')
print(f'max thresh G: {np.max(opt_thresh_glsh)}')
print(f'max thresh L: {np.max(opt_thresh_lof)}')
print(f'max thresh C123: {np.max(opt_thresh_comb123)}')
print(f'max thresh C23: {np.max(opt_thresh_comb23)}')
print(f'min thresh M: {np.min(opt_thresh_mag)}')
print(f'min thresh Med: {np.min(opt_thresh_med)}')
print(f'min thresh H: {np.min(opt_thresh_hdb)}')
print(f'min thresh G: {np.min(opt_thresh_glsh)}')
print(f'min thresh L: {np.min(opt_thresh_lof)}')
print(f'min thresh C123: {np.min(opt_thresh_comb123)}')
print(f'min thresh C23: {np.min(opt_thresh_comb23)}')

fig, ax = plt.subplots(2,3)

auc0 = ax.flatten()[0].imshow(np.flipud(np.mean(auc_mag_mat,axis=2)),vmin=0.5,vmax=1)
auc0 = ax.flatten()[1].imshow(np.flipud(np.mean(auc_med_mat,axis=2)),vmin=0.5,vmax=1)
auc1 = ax.flatten()[2].imshow(np.flipud(np.mean(auc_lof_mat,axis=2)),vmin=0.5,vmax=1)
auc2 = ax.flatten()[3].imshow(np.flipud(np.mean(auc_hdb_mat,axis=2)),vmin=0.5,vmax=1)
auc3 = ax.flatten()[4].imshow(np.flipud(np.mean(auc_glsh_mat,axis=2)),vmin=0.5,vmax=1)
# auc4 = .flatten()ax[0,4].imshow(np.flipud(np.mean(auc_comb123_mat,axis=2)),vmin=0.5,vmax=1)
auc5 = ax.flatten()[5].imshow(np.flipud(np.mean(auc_comb23_mat,axis=2)),vmin=0.5,vmax=1)

ax.flatten()[0].set_title('A: Magnitude thresh.')
ax.flatten()[1].set_title('B: Median difference')
ax.flatten()[2].set_title('C: LOF')
ax.flatten()[3].set_title('D: HDBSCAN')
ax.flatten()[4].set_title('E: GLOSH')
ax.flatten()[5].set_title('F: COMB median HDBSCAN')

cax = add_right_cax(ax.flatten()[5], pad=0.02, width=0.02)
cbar = fig.colorbar(auc5, cax=cax,label='AUC [-]')
# fig.colorbar(auc5, pad = 0.25,label='AUC [-]')

# for a in ax.ravel():
#     a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
#     a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)

ax[0,0].set_axis_off()

ax[0,1].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[0,1].set_yticks([7,6,5,4,3,2,1,0],labels=['2','3','4','5','6','7','8','9'])

ax[0,2].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[0,2].set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)

for a in [ax[1,0],ax[1,1],ax[1,2]]:
    a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
    a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)


ax.flatten()[1].set_ylabel('filter radius [pixels]')
ax.flatten()[2].set_ylabel('knn [-]')
ax.flatten()[3].set_ylabel('min. cluster size [-]')
ax.flatten()[4].set_ylabel('min. cluster size [-]')
ax.flatten()[5].set_ylabel('min. cluster size [-]')


ax.flatten()[3].set_xlabel('min. sample fraction')
ax.flatten()[4].set_xlabel('min. sample fraction')
ax.flatten()[5].set_xlabel('min. sample fraction')
# fig.tight_layout()

fig, ax = plt.subplots(1,6)

auc0 = ax.flatten()[0].imshow(np.flipud(np.mean(auc_mag_mat,axis=2)),vmin=0.5,vmax=1)
auc0 = ax.flatten()[1].imshow(np.flipud(np.mean(auc_med_mat,axis=2)),vmin=0.5,vmax=1)
auc1 = ax.flatten()[2].imshow(np.flipud(np.mean(auc_lof_mat,axis=2)),vmin=0.5,vmax=1)
auc2 = ax.flatten()[3].imshow(np.flipud(np.mean(auc_hdb_mat,axis=2)),vmin=0.5,vmax=1)
auc3 = ax.flatten()[4].imshow(np.flipud(np.mean(auc_glsh_mat,axis=2)),vmin=0.5,vmax=1)
# auc4 = .flatten()ax[0,4].imshow(np.flipud(np.mean(auc_comb123_mat,axis=2)),vmin=0.5,vmax=1)
auc5 = ax.flatten()[5].imshow(np.flipud(np.mean(auc_comb23_mat,axis=2)),vmin=0.5,vmax=1)

ax.flatten()[0].set_title('A: Magnitude thresh.')
ax.flatten()[1].set_title('B: Median difference')
ax.flatten()[2].set_title('C: LOF')
ax.flatten()[3].set_title('D: HDBSCAN')
ax.flatten()[4].set_title('E: GLOSH')
ax.flatten()[5].set_title('F: COMB\nmedian HDBSCAN')

cax = add_right_cax(ax.flatten()[5], pad=0.02, width=0.02)
cbar = fig.colorbar(auc5, cax=cax,label='AUC [-]')

ax[0].set_axis_off()
ax[1].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[1].set_yticks([7,6,5,4,3,2,1,0],labels=['2','3','4','5','6','7','8','9'])
ax[2].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[2].set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)

for a in [ax[3],ax[4],ax[5]]:
    a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
    a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)


ax.flatten()[1].set_ylabel('filter radius [pixels]')
ax.flatten()[2].set_ylabel('knn [-]')
ax.flatten()[3].set_ylabel('minimum cluster size [-]')
ax.flatten()[4].set_ylabel('minimum cluster size [-]')
ax.flatten()[5].set_ylabel('minimum cluster size [-]')


ax.flatten()[3].set_xlabel('min_sample fraction')
ax.flatten()[4].set_xlabel('min_sample fraction')
ax.flatten()[5].set_xlabel('min_sample fraction')
fig.subplots_adjust(left=0.02, bottom=0, right=0.9, top=1, wspace=0.4, hspace=0)


# fig, ax = plt.subplots(4,6)

# auc0 = ax[0,0].imshow(np.flipud(np.mean(auc_mag_mat,axis=2)),vmin=0.5,vmax=1)
# auc0 = ax[0,1].imshow(np.flipud(np.mean(auc_med_mat,axis=2)),vmin=0.5,vmax=1)
# auc1 = ax[0,2].imshow(np.flipud(np.mean(auc_lof_mat,axis=2)),vmin=0.5,vmax=1)
# auc2 = ax[0,3].imshow(np.flipud(np.mean(auc_hdb_mat,axis=2)),vmin=0.5,vmax=1)
# auc3 = ax[0,4].imshow(np.flipud(np.mean(auc_glsh_mat,axis=2)),vmin=0.5,vmax=1)
# # auc4 = ax[0,4].imshow(np.flipud(np.mean(auc_comb123_mat,axis=2)),vmin=0.5,vmax=1)
# auc5 = ax[0,5].imshow(np.flipud(np.mean(auc_comb23_mat,axis=2)),vmin=0.5,vmax=1)

# ax[0,0].set_title('Magnitude thresh.')
# ax[0,1].set_title('Magnitude thresh.')
# ax[0,2].set_title('LOF')
# ax[0,3].set_title('HDBSCAN')
# ax[0,4].set_title('GLOSH')
# # ax[0,4].set_title('COMB123')
# ax[0,5].set_title('COMB23')

# fig.colorbar(auc0, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc1, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc2, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc3, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc4, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc5, pad = 0.25,label='AUC [-]')

# auc_std0 = ax[1,0].imshow(np.flipud(np.std(auc_mag_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std1 = ax[1,1].imshow(np.flipud(np.std(auc_med_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std2 = ax[1,2].imshow(np.flipud(np.std(auc_lof_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std3 = ax[1,3].imshow(np.flipud(np.std(auc_hdb_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std4 = ax[1,4].imshow(np.flipud(np.std(auc_glsh_mat,axis=2)),vmin=0,vmax=0.1)
# # auc_std4 = ax[1,4].imshow(np.flipud(np.std(auc_comb123_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std5 = ax[1,5].imshow(np.flipud(np.std(auc_comb23_mat,axis=2)),vmin=0,vmax=0.1)



# # fig.colorbar(auc_std0, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std1, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std2, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std3, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std4, pad = 0.25, label='AUC std. [-]')
# fig.colorbar(auc_std5, pad = 0.25, label='AUC std. [-]')


# thresh0 = ax[2,0].imshow(np.flipud(np.mean(opt_thresh_mag,axis=2)),vmin=0.5,vmax=1)
# thresh1 = ax[2,1].imshow(np.flipud(np.mean(opt_thresh_med,axis=2)),vmin=0.5,vmax=1)
# thresh2 = ax[2,2].imshow(np.flipud(np.mean(opt_thresh_lof,axis=2)),vmin=0.5,vmax=1)
# thresh3 = ax[2,3].imshow(np.flipud(np.mean(opt_thresh_hdb,axis=2)),vmin=0.5,vmax=1)
# thresh4 = ax[2,4].imshow(np.flipud(np.mean(opt_thresh_glsh,axis=2)),vmin=0.5,vmax=1)
# # thresh4 = ax[2,4].imshow(np.flipud(np.mean(opt_thresh_comb123,axis=2)),vmin=0.5,vmax=1)
# thresh5 = ax[2,5].imshow(np.flipud(np.mean(opt_thresh_comb23,axis=2)),vmin=0.5,vmax=1)

# # fig.colorbar(thresh0,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh1,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh2,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh3,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh4,pad=0.25,label='inlier prob. [-]')
# fig.colorbar(thresh5,pad=0.25,label='inlier prob. [-]')

# std0 = ax[3,0].imshow(np.flipud(np.std(opt_thresh_mag,axis=2)),vmax=0.25)
# std1 = ax[3,1].imshow(np.flipud(np.std(opt_thresh_med,axis=2)),vmax=0.25)
# std2 = ax[3,2].imshow(np.flipud(np.std(opt_thresh_lof,axis=2)),vmax=0.25)
# std3 = ax[3,3].imshow(np.flipud(np.std(opt_thresh_hdb,axis=2)),vmax=0.25)
# std4 = ax[3,4].imshow(np.flipud(np.std(opt_thresh_glsh,axis=2)),vmax=0.25)
# # std4 = ax[3,4].imshow(np.flipud(np.std(opt_thresh_comb123,axis=2)),vmax=0.25)
# std5 = ax[3,5].imshow(np.flipud(np.std(opt_thresh_comb23,axis=2)),vmax=0.25)

# # fig.colorbar(std0,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std1,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std2,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std3,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std4,pad=0.25,label='inlier prob. std. [-]')
# fig.colorbar(std5,pad=0.25,label='inlier prob. std. [-]')


# ax[0,0].set_ylabel('AUC\nmin_clust | knn')
# ax[1,0].set_ylabel('AUC std.\nmin_clust | knn')
# ax[2,0].set_ylabel('opt. threshold\nmin_clust | knn')
# ax[3,0].set_ylabel('opt. threshold std.\nmin_clust | knn')


# ax[3,0].set_xlabel('min_sample fraction')
# ax[3,1].set_xlabel('min_sample fraction')
# ax[3,2].set_xlabel('min_sample fraction')
# ax[3,3].set_xlabel('min_sample fraction')
# # ax[3,4].set_xlabel('min_sample fraction')
# ax[3,4].set_xlabel('min_sample fraction')
# ax[3,5].set_xlabel('min_sample fraction')

# ax[0,0].set_ylabel('min_cluster_size')

# min_samp_label = [ str(i) for i in min_samples_facts]
# min_samp_label[0] = '>0'
# # min_clust_label_facts = [str(i) for i in min_clust_facts]
# min_clust_label_facts = [str(i) for i in min_cluster_sizes]
# min_clust_label = [str(i) for i in min_cluster_sizes]

# for a in ax.ravel():
#     a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
#     a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)
#     # secay = a.secondary_yaxis('right')
#     # secay.set_ylabel('* N_{overlap}')
#     # secay.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label_facts)


# fig.tight_layout()

0
max pos M: 4 : 0.9754902424102132
max pos Med: 50 : 0.9886033136166483
max pos H: 11 : 0.5249359244932238
max pos G: 22 : 0.9930092025607723
max pos L: 38 : 0.985293692579504
max pos C123: 60 : 0.6538033812317682
max pos C23: 66 : 0.4945218367344966
1
max pos M: 4 : 0.9754902424102132
max pos Med: 50 : 0.9886033136166483
max pos H: 1 : 1.0
max pos G: 34 : 0.984180397180421
max pos L: 38 : 0.985293692579504
max pos C123: 54 : 0.9764338860226865
max pos C23: 38 : 0.9943016568083242
2
max pos M: 4 : 0.9754902424102132
max pos Med: 50 : 0.9886033136166483
max pos H: 2 : 0.9972572789346281
max pos G: 31 : 0.9823262061580874
max pos L: 38 : 0.985293692579504
max pos C123: 42 : 0.9224703294925704
max pos C23: 31 : 0.9928508642143612
3
max pos M: 4 : 0.9754902424102132
max pos Med: 50 : 0.9886033136166483
max pos H: 1 : 1.0
max pos G: 29 : 0.9791747470683021
max pos L: 38 : 0.985293692579504
max pos C123: 36 : 0.9814139357854069
max pos C23: 40 : 0.9943016568083242
4
max pos M: 4 : 0.9754902

In [11]:
%matplotlib osx
# import matplotlib as mpl
# mpl.rcParams.update(mpl.rcParamsDefault)
# mpl.rcParams.update({'font.size': 14})

fig, ax = plt.subplots(3,5)
alpha =0.3

for i, obj in enumerate(example_pairs[0].Stack):

    obj_cp = copy.deepcopy(obj)

    # get attr.
    mag = getattr(obj_cp,'Mag')
    med_diff = getattr(obj_cp,'Mag_off_med_diff_7')
    LOF_scores = getattr(obj_cp, f"LOF_outlier_scores_300")
    HDBSCAN_labels = getattr(obj_cp, f"HDBSCAN_probabilities_300_1")
    HDBSCAN_labels2 = getattr(obj_cp, f"HDBSCAN_probabilities_300_30")
    GLOSH_labels = getattr(obj_cp, f"HDBSCAN_outlier_scores_300_1")

    # calculate probablities
    p_in_mag = 1 - (mag-np.nanmin(mag))/(np.nanmax(mag)-np.nanmin(mag))
    p_in_med = 1 - med_diff/(np.nanmax(med_diff))
    p_in_LOF = (LOF_scores-np.nanmin(LOF_scores))/(np.nanmax(LOF_scores)-np.nanmin(LOF_scores))
    p_in_HDBSCAN = HDBSCAN_labels
    p_in_HDBSCAN2 = HDBSCAN_labels2
    p_in_GLOSH = 1 - GLOSH_labels
    p_in_comb23 = (p_in_HDBSCAN+p_in_med)/2 

    # assign as attributes
    setattr(obj_cp,'p_in_mag',p_in_mag)
    setattr(obj_cp,'p_in_med',p_in_med)
    setattr(obj_cp,'p_in_LOF',p_in_LOF)
    setattr(obj_cp,'p_in_HDBSCAN',p_in_HDBSCAN)
    setattr(obj_cp,'p_in_HDBSCAN2',p_in_HDBSCAN2)
    setattr(obj_cp,'p_in_GLOSH',p_in_GLOSH)
    setattr(obj_cp,'p_in_comb23',p_in_comb23)

    q1 = plot_vec_attr_alpha(obj_cp,'p_in_med',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax[0,i]))
    q2 = plot_vec_attr_alpha(obj_cp,'p_in_HDBSCAN',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax[1,i]))
    q3 = plot_vec_attr_alpha(obj_cp,'p_in_comb23',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax[2,i]))

a=1
cax = add_right_cax(ax[0,4], pad=0.02, width=0.02)
cbar = fig.colorbar(q1, cax=cax,label='inlier probability [-]')
cax = add_right_cax(ax[1,4], pad=0.02, width=0.02)
cbar = fig.colorbar(q2, cax=cax,label='inlier probability [-]')
cax = add_right_cax(ax[2,4], pad=0.02, width=0.02)
cbar = fig.colorbar(q3, cax=cax,label='inlier probability [-]')
plt.show()

# fig.tight_layout()

NameError: name 'plot_vec_attr_alpha' is not defined